In [3]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("请输入TAVILY_API_KEY: ")

In [5]:
# 导入相关功能
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# 创建 agent
memory = MemorySaver()
model = ChatOpenAI(
    model="shmily_006/Qw3",
    base_url="http://localhost:11434/v1",
    api_key="EMPTY",
)
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [6]:
# 使用 agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "嗨，我是 Bob，我住在深圳。",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

嗨，我是 Bob，我住在深圳。
================================== Ai Message ==================================

你好，Bob！很高兴认识你。你住在深圳，那里是一个美丽的城市，有繁华的都市生活和丰富的文化。有什么我可以帮你的吗？


In [7]:
input_message = {
    "role": "user",
    "content": "我住的地方天气怎么样？",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

我住的地方天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_dz5na9ep)
 Call ID: call_dz5na9ep
  Args:
    include_domains: ['weather.com', 'forecast.io']
    include_images: True
    query: 深圳的天气情况
    search_depth: advanced
    time_range: year
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "深圳的天气情况", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://weather.com/zh-CN/weather/tenday/l/5f0791acb410c70f6088e6f91c3cfaee1966c75533c46437a76a3cf20569f5ba", "title": "深圳市, 广东省10 天天气预报", "content": "## recents\n\n菜单\n\n## Products & Account\n\n### 特殊天气预报\n\n# 10 天天气-深圳市, 广东省\n\n## 今天白天\n\n## 周六 12 | 白天\n\n雷暴。 最高 31°C。 南 风 15 到 25 每 km / h 。 降雨几率 90%。\n\n## 周六 12 | 夜间\n\n阵雨。 最低 28°C。 南 风 10 到 15 每 km / h 。 降雨几率 60%。\n\n## 周日 13\